# GPT-4o (Extraction 2.1)

In [1]:
from utils.image2base64 import image2base64
from utils.imageWithAxis2Base64 import imageWithAxis2Base64
from utils.getImageSize import getImageSize

from dotenv import load_dotenv, find_dotenv
import os

from langchain_openai import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

In [2]:
pz_path = "../data/processed/bpläne/1_alles_in_einem_dokument/F11-01-PZ-BB.png"
pze_path = "../data/processed/bpläne/1_alles_in_einem_dokument/F11-01-PZE.png"

#base64_pz = imageWithAxis2Base64(pz_path)
base64_pz = image2base64(pz_path)
base64_pze = image2base64(pze_path)

print(getImageSize(pze_path))
print(getImageSize(pz_path))

[768, 1949]
[1359, 768]


In [26]:
msg1 = model.invoke([
    AIMessage(
        content=[
            {
            "type": "text",
            "text": "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
            }
        ]
    ),
    HumanMessage(
        content=[
        {
            "type": "text",
            "text": "Die Zeichenerklärung enthält das Schema der Nutzungsschablone (in Form einer Tabelle). Extrahiere das Schema der Nutzungsschablone in ASCII-Tabellenform."
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_pze}",
                "detail": "high"
            }
        },
        ]
    ),
])

print(msg1.content)

Hier ist das Schema der Nutzungsschablone in ASCII-Tabellenform:

```
+----------------+----------------+
| Art der Nutzung| Gebäudehöhe    |
+----------------+----------------+
| Grundflächenzahl| Geschossflächen-|
|                | zahl           |
+----------------+----------------+
| Bauweise       |                |
+----------------+----------------+
```

Diese Tabelle stellt die verschiedenen Kategorien und deren Beschreibungen dar, die in der Zeichenerklärung des Bebauungsplans verwendet werden.


In [29]:
msg2 = model.invoke([
    AIMessage(
        content=[
            {
            "type": "text",
            "text": "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
            },
            {
                "type": "text",
                "text": f"{msg1.content}"
            }
        ]
    ),
    HumanMessage(
        content=[
        {
            "type": "text",
            "text": 'Fasse relevante Definitionen und Zeichen bezüglich der Schlagwörter aus der ASCII-Nutzungsschablone aus der Zeichenerklärung zusammen. Ausgabe im JSON-Format: {<Schlagwort>: {"Definition"="", "Zeichen": ""}}'
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_pze}",
                "detail": "high"
            }
        },
        ]
    ),
])

print(msg2.content)

```json
{
  "Art der Nutzung": {
    "Definition": "Gewerbegebiete (§ 8 BauNVO), Gewerbegebiete eingeschränkt (§ 8 BauNVO)",
    "Zeichen": "GE, GEE"
  },
  "Gebäudehöhe": {
    "Definition": "max. zulässige Gebäudehöhe über Erdgeschossfußbodenhöhe",
    "Zeichen": "GH=max.12,00"
  },
  "Grundflächenzahl": {
    "Definition": "Grundflächenzahl (GRZ) (§ 16 BauNVO)",
    "Zeichen": "0,6"
  },
  "Geschossflächenzahl": {
    "Definition": "Geschossflächenzahl (GFZ) (§ 16 BauNVO)",
    "Zeichen": "1,2"
  },
  "Bauweise": {
    "Definition": "Abweichende Bauweise (§ 22 BauNVO)",
    "Zeichen": "a"
  },
  "Nutzungsschablone": {
    "Definition": "Nutzungsschablone (§ 9 (1) 1 BauGB)",
    "Zeichen": ""
  }
}
```


In [28]:
msg3 = model.invoke([
    AIMessage(
        content=[
            {
            "type": "text",
            "text": "Du bist ein Assistent um Textinformationen aus einer Planzeichnung eines Bebauungsplans zu extrahieren."
            },
            {
                "type": "text",
                "text": f"{msg1.content}"
            },
            {
                "type": "text",
                "text": f"{msg2.content}"
            }
        ]
    ),
    HumanMessage(
        content=[
        {
            "type": "text",
            "text": "Die Nutzungsschablonen in der vorliegenden Planzeichnung sind durch rote Bounding-Boxen markiert. Gebe die jeweiligen Werte der Nutzungsschablonen im JSON-Format aus."
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{base64_pz}",
                "detail": "high"
            }
        },
        ]
    ),
])

print(msg3.content)

Hier sind die extrahierten Werte der Nutzungsschablonen im JSON-Format:

```json
[
  {
    "Art der Nutzung": "GEE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GEE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "-"
  },
  {
    "Art der Nutzung": "GE",
    "Gebäudehöhe": "max.12,00",
    "Grundflächenzahl": "0,6",
    "Geschossflächenzahl": "1,2",
    "Bauweise": "-"
  }
]
```
